In [10]:
import covalent as ct
from pydantic import BaseModel, Field
import numpy as np
from typing import List

### Define pair interaction potentials

In [8]:
class Potential(BaseModel):
    name: str
    
    def __call__(self, r: float, *args, **kwargs):
        raise NotImplementedError
        
class LennardJones126(Potential):
    name = "lj/126"
    epsilon: float = 1.0
    sigma: float = 1.0
    cutoff: float = 2.5
    
    def __call__(self, r: float):
        return 4*self.epsilon*((self.sigma/r)**12 - (self.sigma/r)**6) if r <= cutoff else 0.0

### Pair force

In [6]:
class Force(BaseModel):
    delta: float = 1e-8
    def __call__(self, r: float, potential: Potential, *args, **kwargs):
        raise NotImplementedError
        
class LennardJones126Force(Force):
    def __call__(self, r: float, potential: LennardJones126):
        return -((potential(r + self.delta) - potential(r - self.delta))/(2*self.delta))

#### Particle base class

In [7]:
class Particle(BaseModel):
    id: int
    mass: float = 1.0
    x: float = Field(default_factory=np.random.uniform)
    y: float = Field(default_factory=np.random.uniform)
    vx: float = Field(default_factory=np.random.uniform)
    vy: float = Field(default_factory=np.random.uniform)
    fx: float = 0.0
    fy: float = 0.0

In [12]:
class TotalPotentialEnergy(BaseModel):
    def __call__(self, particles: List[Particle], potential: Potential, *args, **kwargs):
        pe = 0.0
        for i in particles:
            for j in particles:
                if i == j:
                    continue
                else:
                    dx, dy, dr = i - j
                    pe += potential(dr)
        return pe

#### Create the domain (simulation box)

In [ ]:
domain = Domain2d(xlo = -2.5, xhi=2.5, ylo=-2.5, yhi=2.5)

In [ ]:
particles = [Particle2d(id=i, mass=1.0) for i in range(10)]

In [ ]:
simulation = Simulation(domain=domain, particles=particles, potential=LennardJones126())

In [ ]:
def minimize_pe(total_pe, particles: List[Particle], *args, **kwargs):
    

In [ ]:
simulation.total_pe

In [ ]:
Simulation(domain=domain)

In [ ]:
@ct.electron
class Temp(object):
    def __call__(self, r: float):
        return r**2

In [ ]:
@ct.lattice
def workflow(r):
    return Temp(r)

In [ ]:
res = ct.dispatch_sync(workflow)(2.0)

In [ ]:
potential = LennardJones126()(2.0)

In [ ]:
potential

In [ ]:
p = Particle2d(id=1, mass=1.0)